In [1]:
import pandas as pd
import geopandas as gpd
import networkx as nx
import numpy as np
from datetime import datetime
from dateutil import parser
# pip install azureml-opendatasets-runtimeusing
from azureml.opendatasets import NycTlcYellow
import calendar
import numpy.linalg as linalg
import matplotlib.pyplot as plt
import pickle
import momepy

In [ ]:
# Only need to run this once ever
def preprocess_lion():
    # Download data from https://www.dropbox.com/sh/927yoof5wq6ukeo/AAA--Iyb7UUDhfWIF2fncppba?dl=0
    # Put all files into 'data_unwrangled/LION' or change path below
    lion_filename = 'data_unwrangled/LION/links.shp'
    new_folder = 'data/infrastructure/'
    # Load all LION data
    links = gpd.read_file('data_unwrangled/LION/links.shp')
    # Only consider links in Manhattan
    links = links[links['LBoro']==1]
    # Only consider links that are normal streets
    links = links[links['FeatureTyp']=='0']
    # Only consider constructed links
    links = links[links['Status']=='2']
    # Ensure *undirected* graph is connected
    # Note: We could do this for directed graph but maximum size
    # of strongly connected component is 430
    graph = momepy.gdf_to_nx(links, approach="primal", directed=False)
    for component in nx.connected_components(graph):
        if len(component) > 10000:
            graph = graph.subgraph(component)
    # Use resulting links and nodes as infrastructure
    nodes, links = momepy.nx_to_gdf(graph)
    # Save both nodes and links
    nodes.to_file(new_folder + 'nodes.json', driver='GeoJSON')
    links.to_file(new_folder + 'links.json', driver='GeoJSON')

In [48]:
def load_filter():
    filename_filter = 'data_unwrangled/2010 Neighborhood Tabulation Areas (NTAs).geojson'
    filter = gpd.read_file(filename_filter)
    filter = filter[filter['boro_name'] == 'Manhattan']
    return filter

def preprocess_taxi(df):
    # Make sure rides are longer than one minute
    df = df[df['tpepDropoffDateTime'] - df['tpepPickupDateTime'] > np.timedelta64(1, 'm')]
    # Make sure rides are shorter than 12 hours
    df = df[df['tpepDropoffDateTime'] - df['tpepPickupDateTime'] <= np.timedelta64(12, 'h')]
    # Make sure rides are longer than .1 mile
    df = df[df['tripDistance'] > 0.1]
    # Make sure fare is non-zero 
    df = df[df['fareAmount'] > 0.0]
    # Convert to geopandas
    gdf = gpd.GeoDataFrame(df)
    # Reset index ID (there are duplicate indices)
    gdf.reset_index(inplace=True)
    # Create ride ID
    gdf['ride_id'] = gdf.index
    # Make start time date time type
    gdf['start_time'] = pd.to_datetime(gdf['tpepPickupDateTime'])
    # Round start time to day
    gdf['start_day'] = gdf['start_time'].dt.round('d')
    return gdf

def filter_location(type, filter, taxi, make_copy=True):
    # Create a geometry column from the type coordinates
    taxi[f'{type}_geom'] = gpd.points_from_xy(taxi[f'{type}Lon'], taxi[f'{type}Lat'])
    taxi.set_geometry(f'{type}_geom', inplace=True)
    taxi = taxi.sjoin(filter)

    # Drop the index_right column from sjoin
    if 'index_right' in taxi.columns:
        taxi.drop(columns=['index_right'], inplace=True)

    return taxi

def restrict_start_end(taxi, check_ratio=False):        
    # Load Manhattan objects
    filter_manhattan = load_filter()
    # Restrict to rides that start in Manhattan
    taxi_start = filter_location('start', filter_manhattan, taxi)
    # Restrict to rides that start and end in Manhattan
    taxi_start_end = filter_location('end', filter_manhattan, taxi_start)
    if check_ratio:
        # Check number of rides that start AND end in Manhattan / number of rides that start OR end in Manhattan
        taxi_end = filter_location('end', filter_manhattan, taxi)
        print(len(taxi_start_end)/(len(taxi_start)+len(taxi_end)-len(taxi_start_end))) # About 85%
    return taxi_start_end

def get_taxi_data(year, month):
    # Get query for first and last day of month in year
    month_last_day = calendar.monthrange(year=int(year),month=int(month))[1]
    start_date = parser.parse(str(year)+'-'+str(month)+'-01')
    end_date = parser.parse(str(year)+'-'+str(month)+'-'+str(month_last_day))
    end_date = parser.parse(str(year)+'-'+str(month)+'-02')
    print('Loading taxi data...', end=' ')
    nyc_tlc = NycTlcYellow(start_date=start_date, end_date=end_date)
    taxi_all = nyc_tlc.to_pandas_dataframe()
    print('complete!')
    print('Preprocessing data...', end=' ')
    taxi = preprocess_taxi(taxi_all)
    print('complete!')
    print('Restricting start and end...', end=' ')
    taxi_start_end = restrict_start_end(taxi)
    print('complete!')

    return taxi_start_end



In [50]:
taxi = get_taxi_data('2013', '7')

In [ ]:
links = gpd.read_file('data/infrastructure/links.json')
nodes = gpd.read_file('data/infrastructure/nodes.json')